<a target="_blank" href="https://colab.research.google.com/github/weaviate/recipes/blob/main/weaviate-features/similarity-search/similarity_search_aws_sagemaker.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Dependencies

In [ ]:
!pip install -U weaviate-client -q

In [ ]:
import weaviate
weaviate.__version__

## Connect to Weaviate

In [ ]:
import weaviate
from weaviate.classes.init import Auth
import os

# Best practice: store your credentials in environment variables
weaviate_url = os.environ["WEAVIATE_URL"]
weaviate_api_key = os.environ["WEAVIATE_API_KEY"]
aws_access_key = os.getenv("AWS_ACCESS_KEY")
aws_secret_key = os.getenv("AWS_SECRET_KEY")

headers = {
    "X-AWS-Access-Key": aws_access_key,
    "X-AWS-Secret-Key": aws_secret_key,
}

# Option 1: Connect to your Weaviate Client Service cluster
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
    headers=headers
)

# Option 2: Connect to an embedded instance
# client = weaviate.connect_to_embedded(
#  headers= headers
#)

# Option 3: Connect to your local Weaviate instance deployed with Docker
# client = weaviate.connect_to_local(
#  headers= headers
#)

print(client.is_ready())
print(client.get_meta()['version'])

## Create a collection
> Collection stores your data and vector embeddings.

In [ ]:
# Note: in practice, you shouldn"t rerun this cell, as it deletes your data
# in "JeopardyQuestion", and then you need to re-import it again.
import weaviate.classes.config as wc
from weaviate.classes.config import Configure

# Delete the collection if it already exists
if (client.collections.exists("JeopardyQuestion")):
    client.collections.delete("JeopardyQuestion")


client.collections.create(
    name="JeopardyQuestion",
    vectorizer_config=[
            Configure.NamedVectors.text2vec_aws(
            name="question",
            region="eu-north-1",
            source_properties=["Question"],
            service="sagemaker",
            endpoint="tei-xxxx", # sagemaker endpoint name "tei-xxxx"
            )
        ],

    properties=[ # defining properties (data schema) is optional
        wc.Property(name="Question", data_type=wc.DataType.TEXT),
        wc.Property(name="Answer", data_type=wc.DataType.TEXT),
        wc.Property(name="Category", data_type=wc.DataType.TEXT, skip_vectorization=True),
    ]
)

print("Successfully created collection: JeopardyQuestion.")

## Import the Data

In [ ]:
import requests, json
url = "https://raw.githubusercontent.com/weaviate/weaviate-examples/main/jeopardy_small_dataset/jeopardy_tiny.json"
resp = requests.get(url)
data = json.loads(resp.text)

# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

# Insert data objects
response = jeopardy.data.insert_many(data)

# Note, the `data` array contains 10 objects, which is great to call insert_many with.
# However, if you have a milion objects to insert, then you should spit them into smaller batches (i.e. 100-1000 per insert)

if (response.has_errors):
    print(response.errors)
else:
    print("Insert complete.")

## Query Weaviate: Similarity Search (Text objects)

Similarity search options for text objects in **Weaviate**:

1. [near_text](https://weaviate.io/developers/weaviate/search/similarity#an-input-medium)

2. [near_object](https://weaviate.io/developers/weaviate/search/similarity#an-object)

3. [near_vector](https://weaviate.io/developers/weaviate/search/similarity#a-vector)

### nearText Example

Find a `JeopardyQuestion` about "animals in movies". Limit it to only 4 responses.

In [ ]:
# note, you can reuse the collection object from the previous cell.
# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

response = jeopardy.query.near_text(
    query="african beasts",
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

Return vector embeddings.

In [ ]:
response = jeopardy.query.near_text(
    query="african beasts",
    include_vector=True,
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2))
    print("Vector:", item.vector, "\n")

Now, also request the `distance` for each returned item.

In [ ]:
import weaviate.classes.query as wq

response = jeopardy.query.near_text(
    query="african beasts",
    return_metadata=wq.MetadataQuery(distance=True),
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Distance:", item.metadata.distance)
    print("Data:", item.properties, "\n")

### nearObject Example

Search through the `JeopardyQuestion` class to find the top 4 objects closest to id `2559fb76-430f-41de-91b4-629967311bb9`. (The id was taken from the query above)

In [ ]:
response = jeopardy.query.near_object(
    near_object="2559fb76-430f-41de-91b4-629967311bb9", # replace with your id of interest
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", item.properties, "\n")

### nearVector Example

Search through the `JeopardyQuestion` class to find the top 2 objects closest to the query vector `[-0.0125526935, -0.021168863, ... ]`

In [ ]:
response = jeopardy.query.near_vector(
    near_vector=[-0.0125526935, -0.021168863, ... ], # your vector object goes here
    limit=4
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", item.properties, "\n")